In [375]:
import json
import os
import re
from itertools import product
from typing import Dict

import numpy
from pandas import DataFrame, MultiIndex, concat
PROCESS_COLUMN = ["Economy", 'Categories', 'Defense', 'General', 'Intel', 'Wreckage','Weapon','Veteran','Transport','Buffs']
STOP_FILE_NAMING = r"\w+(?=_unit)"


def read_json(file_path="../../data/test"):
    data = []
    for root, _, files in os.walk(file_path):
        for _file in files:
            if "json" in _file:
                with open(os.path.join(root, _file), 'r') as f:
                    unit_id = re.findall(STOP_FILE_NAMING, os.path.basename(os.path.join(root, _file)))[0]
                    data.append((unit_id,json.load(f)))
    return data

def build_multiple_index(raw_data:Dict)->MultiIndex:
    def dict_to_tuple(work_dict:Dict, t_key=None):
        res = []
        for key, value in work_dict.items():
            if isinstance(value, dict):
                res += [(t_key, *_) for _ in  dict_to_tuple(value, key)] if t_key else dict_to_tuple(value, key)
            elif t_key:
                res.append((t_key, key))
            else:
                res.append((key,))
        return res
    return MultiIndex.from_tuples(dict_to_tuple(raw_data))

list_unit_id = ["XSL0105"]
build_multiple_index({"a":"v", "b":{"a":"v","y":"x", "c":{"a":"v"}}})



MultiIndex([('a', nan, nan),
            ('b', 'a', nan),
            ('b', 'y', nan),
            ('b', 'c', 'a')],
           )

In [339]:
#todo asked on stackoverflow
columns = MultiIndex.from_tuples(
    # [("t0",),("a","t1"),("a","t2"),("a", "b", "t3"),("a", "b", "c", "t4")],
    [("a","t0"),("b","t1"),("b","t2"),("c", "t1"),("c", "t2")],
                     # names=["lvl1","lvl2","lvl3","lvl4"])
                     names=["lvl1","lvl2"])
df = concat([DataFrame({"df1":numpy.random.randint(0,2,6)}),
             DataFrame({"df2_1":numpy.random.randint(2,4,6),
                        "df2_2":numpy.random.randint(4,6,6)}),
             DataFrame({"df4":numpy.random.randint(6,7,6)}),
             DataFrame({"df5":numpy.random.randint(7,8,6)}),], axis=1)
df.columns = columns
# df.plot.line()
df.loc[:,[("a","t0"),("c","t2")]]
# df
#
# idx = MultiIndex.from_tuples([(1, 'one', "t"), (1, 'two', "t"),
#                                     (2, 'one', "t"), (2, 'two', "t"),
#                                     (3, 'one', "t"), (3, 'two', "t")],
#                                     names=['foo', 'bar', 'new'])
# print(idx.set_levels([["a","b","c"], [1,2,3], ["d","f","g"]],level=[1,2,3]))
# df = DataFrame(data, columns=PROCESS_COLUMN, index=list_unit_id)

lvl1,a,c
lvl2,t0,t2
0,1,7
1,1,7
2,0,7
3,0,7
4,1,7
5,0,7


In [219]:
dfs = []
for data, n in product(read_json(), PROCESS_COLUMN):
    _id = data[0]
    content = data[1]
    for d in content:
        if n in d.keys():
            if isinstance(d[n], dict):
                records:DataFrame = DataFrame(d[n], index=[_id])
                from_product = MultiIndex.from_product([[n.lower()], d[n].keys()])
                records.columns = from_product
                dfs.append(records)
            elif isinstance(d[n],list):
                s = DataFrame([[d[n]]], index=[_id])
                s.columns = MultiIndex.from_product([[n.lower()], ["value"]])
                dfs.append(s)
t = concat(dfs, axis=1)
print(t.columns)
print(dfs[0])

MultiIndex([(   'economy',         'BuildCostEnergy'),
            (   'economy',           'BuildCostMass'),
            (   'economy',               'BuildRate'),
            (   'economy',               'BuildTime'),
            (   'economy',        'MaxBuildDistance'),
            (   'economy', 'NeedToFaceTargetToBuild'),
            (   'economy',           'StorageEnergy'),
            (   'economy',             'StorageMass'),
            (   'economy',       'TeleportEnergyMod'),
            (   'economy',         'TeleportMassMod'),
            (   'economy',         'TeleportTimeMod'),
            (   'economy',       'BuildableCategory'),
            ('categories',                   'value'),
            (   'defense',          'AirThreatLevel'),
            (   'defense',               'ArmorType'),
            (   'defense',      'EconomyThreatLevel'),
            (   'defense',                  'Health'),
            (   'defense',               'MaxHealth'),
          